# Lecture 4
## Making classes in Python

In this lecture we will look at more example of defining classes in Python. We also touch briefly on inheritance, which we will return to look more at later.

### A 3D vector class

In the first part of this lecture, we will work on implementing a class for three dimensional vectors. We do this as an example for different types of special methods and behavior we can include.

#### The constructor (`__init__`)

Recall that the constructor of the class is defined using the init special method (`__init__`). For a 3D vector, we need the three components of the vector. We call these x, y, z, assuming we are working with cartesian coordinates.

In [1]:
import numpy as np

class Vector3D:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z

##### Pretty printing

Next we want to be able to print out vectors, so that we can check our results. We implement this using the string special method (`__str__`). If we were just implementing this in an editor, we would just keep editing our file. In the notebook, we don't want to repeat the whole class for each cell. Instead we just extend our existing class as follows:

In [2]:
class Vector3D(Vector3D):
    def __str__(self):
        return "({:g}, {:g}, {:g})".format(self.x, self.y, self.z)
    
u = Vector3D(0, 4, -2)
print(u)

(0, 4, -2)


Here, extending the class by writing `Vector3D(Vector3D)` works because of *inheritance*, as we will explain later.

Our string special method works nicely, but if we print a list of vectors, the output won't be too nice, so we also add a representation method (`__repr__`).

In [3]:
class Vector3D(Vector3D):
    def __repr__(self):
        return "{}({}, {}, {})".format(self.__class__.__name__,
                                       self.x, self.y, self.z)

vectors = [Vector3D(2, 0, 2), Vector3D(-1, 1, -1), Vector3D(1, 1, 1)]
print(vectors)

[Vector3D(2, 0, 2), Vector3D(-1, 1, -1), Vector3D(1, 1, 1)]


Here we could have also just written out the name of the class explicitly: `return "Vector3D({}, {}, {}).format(...)"`, but this is unwise, because if we change the name of our class we break the `repr`, or if we make a subclass, it will also be broken for the subclass. So instead, we use `self.__class__.__name__`, as all classes know their own name.

### Arithmetic

We want to be able to add or subtract our vector objects. We could do this by simply creating a `add` method, and write for example `u.add(v)`, but this isn't that elegant. Instead, we want to be able to just write `u + v`. When we write out a statement like that, we are using the *binary* operator `+`. When we write `a + b` for any objects `a` and `b`, Python calls the addition special method behind the scenes, so it turns into the call `a.__add__(b)`. 

The result of adding two 3D vectors together is a new 3D vector, so our `__add__` special method should return a new `Vector3D`-object. If this is confusing, imagine the following code snippet
```Python
u = Vector3D(2, 0, -2)
v = Vector3D(2, 4, 2)
w = u + v
```
In this case, adding `u` and `v` to define `w` shouldn't change the values of `u` and `v`, but should instead create a *new vector object* which we refer to as `w`. If this is the case, we need our addition to both create this new object (meaning we need to call the `Vector3D`-constructor) and return it, so that `w` knows what object to refer to.

In [4]:
class Vector3D(Vector3D):
    def __add__(self, other):
        x = self.x + other.x
        y = self.y + other.y
        z = self.z + other.z
        return Vector3D(x, y, z)

u = Vector3D(2, 0, -2)
v = Vector3D(2, 4, 2)
w = u + v

print("{} + {} = {}".format(u, v, w))

(2, 0, -2) + (2, 4, 2) = (4, 4, 0)


#### `self` and `other`

We just defined our addition method with the statement `def __add__(self, other)`, which can be a bit confusing.  To better understand this, it can be helpful to first understand why there must be two arguments. This is because we are talking about *binary* addition, meaning two things are added together. When we write `a + b`, these two objects are being added, so `self` will refer to `a` and `other` will refer to `b`. This can become more clear if we realize what calls are being made behind the scenes:

1. We write: `a + b`
2. Which Python interprets as: `a.__add__(b)`
3. Which in turn actually means: `Vector3D.__add__(a, b)`

So in the method we have written, which has the signature `__add__(self, other)`, we see that `a` is the self, and `b` is the other.

The use of `self` and `other` can be a tricky concept to wrap your head around, but it is fundamental to mastering classes, so it is very worth the time to take time to properly digest this material.

#### Vector-scalar addition

We have added a method to add two vectors together, so we can write `u + v`. What happens if we try to add a vector and an integer `u + 1`?
```Python
print(u + 1)
```
We can an error: `AttributeError: 'int' object has no attribute 'x'`.

This error happens because our `other`-object in this case is no longer a `Vector3D`-object, but an `int`. Here we get an `AttributeError` because we try to use `other.x`, but the integer has no `.x` attribute to access. In this case, we should actually be checking what type `other` and treat it differently depending on its type. We know `self` is a `Vector3D`, so we don't need to check this.

Now, what behavior do we want if actually try to add an integer? We could either say we add the number to each component seperately, so:
```
if isinstance(other, (int, float)):
    self.x += other
    self.y += other
    self.z += other
```
This is how a numpy array works for example. In our case, however, we want the `Vector3D` class to represent a mathematical vector, and for these, adding a vector and a scaler together doesn't make sense. Therefore, we throw an exception instead, we generalize this by throwing an error if we try to add anything other than another vector object.

In [5]:
class Vector3D(Vector3D):
    def __add__(self, other):
        if isinstance(other, Vector3D):
            x = self.x + other.x
            y = self.y + other.y
            z = self.z + other.z
            return Vector3D(x, y, z)
        
        else:
            raise TypeError("cannot add vector and {}".format(type(other)))


u = Vector3D(2, 0, -2)
v = Vector3D(2, 4, 2)

print("{} + {} = {}".format(u, v, u + v))

try:
    print("{} + {} = {}".format(u, 1, u + 1))
except TypeError as e:
    print("{} + {} = {}".format(u, 1, e))

(2, 0, -2) + (2, 4, 2) = (4, 4, 0)
(2, 0, -2) + 1 = cannot add vector and <class 'int'>


#### Subtraction

Extending our class to also handle subtraction of vectors is very similar to addition, we simply use the subtraction special method instead (`__sub__`).

In [6]:
class Vector3D(Vector3D):
    def __sub__(self, other):
        if isinstance(other, Vector3D):
            x = self.x - other.x
            y = self.y - other.y
            z = self.z - other.z
            return Vector3D(x, y, z)
        
        else:
            raise TypeError("cannot subtract vector and {}".format(type(other)))
            
            
u = Vector3D(2, 0, -2)
v = Vector3D(2, 4, 2)

print("{} + {} = {}".format(u, v, u + v))
print("{} - {} = {}".format(u, v, u - v))

try:
    print("{} + {} = {}".format(u, 4, u + 4))
except TypeError as e:
    print("{} + {} = {}".format(u, 4, e))

(2, 0, -2) + (2, 4, 2) = (4, 4, 0)
(2, 0, -2) - (2, 4, 2) = (0, -4, -4)
(2, 0, -2) + 4 = cannot add vector and <class 'int'>


From mathematics, we know that `u - (-v)` is equivalent to `u + v`, but does Python understand this? You can try yourself and find that, no, it does not. This is because Python tried to resolve what we write inside the parenthensis *first*, which is `(-u)`. However, Python doesn't understand what the negative of a vector is. This is reflected in the error message you get:
```
TypeError: bad operand type for unary -: 'Vector3D'
```
Here "unary -" is reffering to the fact that `-u` is a *unary* operator, where as `u - v` is a *binary* operator. Unsurprisingly, there is a special method for the unary negative, it is called `__neg__`. Because it is unary, we define it without the "other" argument:

In [7]:
class Vector3D(Vector3D):
    def __neg__(self):
        return Vector3D(-self.x, -self.y, -self.z)

u = Vector3D(2, 0, -2)
v = Vector3D(2, 4, 2)

print("{} - (-{}) = {}".format(u, v, u - (-v)))

(2, 0, -2) - (-(2, 4, 2)) = (4, 4, 0)


If we had implemented our `__neg__` first, it could have made our `__sub__` method easy to implement:

In [8]:
class Vector3D(Vector3D):
    def __sub__(self, other):
        if isinstance(other, Vector3D):
            return self + (-other)
        else:
            raise TypeError("cannot subtract vector and {}".format(type(other)))
        
u = Vector3D(2, 0, -2)
v = Vector3D(2, 4, 2)

print("{} - {} = {}".format(u, v, u - v))

(2, 0, -2) - (2, 4, 2) = (0, -4, -4)


#### Multiplication vectors

We now want to add functionality for multiplying vectors. For 3D vectors, several type of multiplication exists. We want to be able to do both the dot product, and the cross product. Let us implement these as normal methods (not special methods) first.




In [9]:
class Vector3D(Vector3D):
    def dot(self, other):
        return self.x*other.x + self.y*other.y + self.z*other.z
    
    def cross(self, other):
        x = self.y*other.z - self.z*other.y
        y = self.z*other.x - self.x*other.z
        z = self.x*other.y - self.y*other.x
        return Vector3D(x, y, z)

If we also want to be able to use these multiplications as special methods, we can now simply call `dot` or `cross` in our special methods. Let us say we decide we want `u*v` to mean dot product, and `u@v` to mean cross product, then we simply add the two special methods (`__mul__` and `__matmul__`) as follows:

In [10]:
class Vector3D(Vector3D):
    def __mul__(self, other):
        """Interpret u*v to be the dot product"""
        return self.dot(other)
    
    def __matmul__(self, other):
        """Interpret u@v as cross product"""
        return self.cross(other)

In [11]:
u = Vector3D(1, -1, 0)
v = Vector3D(1, 4, 2)

print(u*v)
print(u@v)

-3
(-2, -2, 5)


We can now also add a method for checking if two vectors are perpendicular, we do this by checking if $u\cdot v = 0$.

In [12]:
class Vector3D(Vector3D):
    def perpendicular(self, other):
        return abs(self*other) < 1e-9

u = Vector3D(1, -1, 0)
v = Vector3D(1, 4, 2)
print(u.perpendicular(v))

False


And we know that if $w = u \times v$, then $w \perp u$ and $w \perp v$. Let us check:

In [13]:
w = u@v
print(w.perpendicular(u))
print(w.perpendicular(v))

True
True


#### Scalar multiplication

We can also multiply vectors by scalars, which is equivalent to multiplying each component by the scalar seperately. We want `u*3` or `3*u` to be how this is done, so we rewrite our `__mul__`-method to do different things depending on the types. This is known as *overloading* an operator, in this case we overload `*` to mean either scalar or dot product, depending on context.

In [14]:
class Vector3D(Vector3D):
    def __mul__(self, other):
        if isinstance(other, Vector3D):
            return self.dot(other)
        elif isinstance(other, (int, float)):
            return Vector3D(self.x*other, self.y*other, self.z*other)
        else:
            raise TypeError("cannot multiply vector and {}".format(type(other)))
            
u = Vector3D(1, -1, 0)
v = Vector3D(1, 4, 2)

print(u*v)
print(u*3)

-3
(3, -3, 0)


So we see that `u*v` is now the dot product of two vectors, and `u*3` is the scalar multiplication. However, what about if we do:
```Python
3*u
```
This leads to an error:
```
TypeError: unsupported operand type(s) for *: 'int' and 'Vector3D'
```
This happens because `3*u` is interpreted behind the scenes as `3.__mul__(u)`, and so it is the multiplication special method of the `int` class that is called! Now, we have a work around, we can simply always write the vector first. However, this isn't that nice, because `3*u` is closer to mathematical convention than writing the vector first. We also want to give our user flexibility. To fix this, we should add a new special method called `__rmul__` (for right multiplication). This method is called if the first normal call of `mul` throws a TypeError. So in this case, we write `3*u`, and Python tries to call `3.__mul__(u)`. This doesn't work, so instead it calls `u.__rmul__(3)`.


In [15]:
class Vector3D(Vector3D):
    def __rmul__(self, other):
        return self*other
    
u = Vector3D(4, -6, 2)
print(2*u)

(8, -12, 4)


This brings up an important point about operators in programmering. While some operators are generally considered commutative in mathematics in math, meaning $a+b = b+a$, we see that this is not the case in Python. We could for example add different function in the `__add__`-method and in the `__radd__` method, and then `a+b` and `b+a` would behave differently. In fact, you might already know several examples of this, for example adding lists:

In [16]:
a = [1, 2, 3]
b = [4, 5, 6]

print(a+b)
print(b+a)

[1, 2, 3, 4, 5, 6]
[4, 5, 6, 1, 2, 3]


#### Length and unit vectors

Often we are interested in the length of a vector, or we want a unit vector with the same orientation as a given vector, let us add this as well.

In Python there is a length special method (`__len__`), that is called by Python when we write `len(u)`. However, this method has to return an integer, as it is mainly meant to find the number of elements in a container, like a list. We therefore ignore this special method and instead implement the length as a property.

In [17]:
class Vector3D(Vector3D):
    @property
    def length(self):
        return np.sqrt(self*self)
    
    @length.setter
    def length(self, new_length):
        scale = new_length/self.length
        self.x *= scale
        self.y *= scale
        self.z *= scale
        
u = Vector3D(2, -2, 1)
print(u, u.length)

u.length = 1
print(u, u.length)

(2, -2, 1) 3.0
(0.666667, -0.666667, 0.333333) 1.0


We now want to create a metho `unit` that returns a unit vector of our given vector. So that we can do
```
u = Vector3D(2, -2, 1)
w = u.unit()
```
Here we intend the method to return a new object, of unit length, and that it shouldn't change the original object. To write this function we can first *copy* the vector, and then change the length of the copied vector:

In [18]:
class Vector3D(Vector3D):
    def unit(self):
        new_vector = Vector3D(self.x, self.y, self.z)
        new_vector.length = 1
        return new_vector

u = Vector3D(2, -2, 1)
w = u.unit()
print(u)
print(w)
print(u is w)

(2, -2, 1)
(0.666667, -0.666667, 0.333333)
False


Writing `u is w` checks wether `u` and `w` are the same object, and we get false, as expected. In this case, making a copy was fairly easy, however, for larger, more complex objects, it can be some work. An alternative then is to use `copy.deepcopy`, which generates a deep copy of an object (`copy.copy` generates a shallow copy). 

#### Other Special methods

Here we have shown different special methods, but there are plenty left to cover. Virtually any possible behavior of an object can be decided through good use of special methods. For a more detailed walkthrough, see [this guide](http://www.diveintopython3.net/special-method-names.html).

Here are some that might be of special interest:
* `__eq__`, for defining when two objects should be considered equal
* We can also implement `lt`, `le`, `gt`, `ge`, `ne` for $\lt, \le, \gt, \ge, \ne$. Implementing all of these is called *rich comparisons*, and automatically makes our objects sortable with `sort()`
* `__bool__` defines what `if x:` means for our object
* `__abs__`, defines what `abs(x)` means
* `__getitem__` and `__setitem__` can be used to make our object indexable: `x[0]`

### Short introduction to inheritance

We will cover inheritance more in detail next week, but for now, let's just show some simple examples. We can let one class *inherit* from another. When we do this, the daughter class gets all functionality, i.e., all attributes, of the parent class. The daughter class is also known as the subclass, and the parent class the superclass. We can also implement new methods for the daughter class, or we can overwrite existing classes. Because of this, making subclasses is a way to specialize a class, while the superclass is a more *general* case.

Let us look at a somewhat silly example: Pokémon Go. If you've played the game, this is hopefully easy to follow, if you haven't, don't worry to much about the specific calculations, but try to focus more on the big picture code.

#### Pokémon Go

In the game Pokémon Go, players catch a bunch of different kinds of Pokémons. Each pokemon caught has different stats, and players want to catch pokemon with good stats. When the game generates a new pokemon, it must also find its stats. And this is what we will write code to do now.

The stats of a specific pokemon is dependent on its species, because each species has specific base stats, in addition each individual pokemon caught has a randomized bonus to its stat, called the *individual value*. These individual values are random numbers between 0 and 15 for all pokemon, regardless of species.

Because some properties vary by species, and some do not, we want to use inheritance. We now want to create a `Pokemon` superclass that has all functionality that is general to all `Pokemon`, and then we can implement specific species as subclasses afterwards.

We now make the `Pokemon` class. Each time a new pokemon is generated, we must draw its individual values randomly. There are three seperate stats: `ATK`, `DEF` and `STA`. We implement these as properties.

In [19]:
import numpy as np

class Pokemon:
    def __init__(self):
        self.IV_ATK = np.random.randint(16)
        self.IV_STA = np.random.randint(16)
        self.IV_DEF = np.random.randint(16)

    @property
    def ATK(self):
        return self.BASE_ATK + self.IV_ATK
    
    @property
    def DEF(self):
        return self.BASE_DEF + self.IV_DEF
    
    @property
    def STA(self):
        return self.BASE_STA + self.IV_STA
    
    def __str__(self):
        return "{}({}, {}, {})".format(self.__class__.__name__,
                                       self.ATK, self.DEF, self.STA)

We now have a class `Pokemon`, but we cannot really use it, because if we try to access any of the three stat properties we get an error, as the base stats are not defined. This is not a bug, its on purpose. We do not want to use the general superclass, as all pokemon in the actual game has a specific species. We state this by saying the `Pokemon` class is an *abstract* class.

Now, let us implement a few specific Pokemon types. Lets start with the most iconic one:

In [20]:
class Pikachu(Pokemon):
    BASE_ATK = 112
    BASE_DEF = 101
    BASE_STA = 70
    
class Charizard(Pokemon):
    BASE_ATK = 223
    BASE_DEF = 176
    BASE_STA = 156

Here we write `Pikachu(Pokemon)`, this syntax means the class is inheriting from the class within the parenthenses and becomes a subclass. We define the base stats as *class attributes* because they are the same for all pikachus, more on class attributes next week.

Now we have very easily made two distinct species of Pokemon, without having to reimplement their general behavior. Inheritance has simplified our overall code considerably.

In [21]:
for i in range(5):
    pkmn = Pikachu()
    print(pkmn)

print()

for i in range(5):
    pkmn = Charizard()
    print(pkmn)

Pikachu(120, 105, 72)
Pikachu(114, 114, 72)
Pikachu(116, 109, 83)
Pikachu(122, 101, 74)
Pikachu(127, 106, 73)

Charizard(229, 181, 168)
Charizard(223, 190, 160)
Charizard(226, 182, 158)
Charizard(230, 181, 165)
Charizard(226, 185, 162)


That was it for this example. The point was to illustrate how the goal of inheritance is to put all the general behavior in the superclass, and then only implement the *specifics* in the subclasses to specialize them.

If you are interested in Pokemon Go, you can now extend the Pokemon class to for example calculate the CP, and so on.